## SETI end-to-end narrowband drifting tone detection over multiple raw files
This notebook sequences through multiple raw files and looks for drifting narrowband tones.  

This is a general-purpose notebook for real observations such as Voyager or commensal pointings.

For each test file, notebook "01_seti_end_to_end.ipynb" is called which performs the following:
1) Inputs each RAW file 
2) Runs Rawspec to generate a filterbank .h5 spectrogram file with specified fine FFT size and integration factor n_sti
3) Runs TurboSETI and/or seticore and compiles a list of detections and compute time (wall clock)

Note that these sets of files can be done with various branches of seticore or seticore2 and detection SNR or timing comparisons
can be made between branches over multiple runs of this notebook.  For each run of this notebook, 
the desired seticore branch needs to be checked out and compiled, and 
the string "test_case" needs to be set in 01_seti_end_to_end.ipynb to indicate the branch.

The raw files are copied locally from bulk storage if needed, or rawspec can be run directly from bulk storage without copying locally. 
If local space is limited, raw files can be deleted after each run of this notebook. 

Plots are generated in each "01_seti_end_to_end.ipynb" call.

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

parameters_are_undefined = False     # use parameter defaults settings in 01_seti_end_to_end.ipynb

app_to_run = 'seticore-10'
# app_to_run = 'seticore-20'
# app_to_run = 'bliss-10-8'
# app_to_run = 'bliss-10-12'

delete_raw_file_after_use = True
display_figs01 = False
plot_all_dets = True
plot_multichirp_dets1 = True
plot_multichirp_dets2 = False
verbose = True
plot_sg_psd_zoom = True
plot_psd_zoom = False
plot_sg_zoom = False
plot_sg_wb = True
plot_psd_wb = True
zoom_bw_MHz = .012  # +/- 6 KHz
max_n_zoom_plots = 50

run_turbo = False
fb_ext = '.h5'

search_max_drift = 10    # Hz/sec
search_min_drift = -10   # ignored except for plotting

delete_raw_file_enable = True   # =True will delete raw file from $RAWDIR, if not present initially in $RAWDIR,
                                # and copied from $RAW_BACKUP_BASE_DIR/raw_backup_subdir
                                # =False will keep raw file in $RAWDIR regardless
delete_h5_sg_file = True        # =True h5 sg file will be deleted after finishing (reduce file space in $SGDIR)

enable_raw_backup_copy = False  # =True copy to $RAWDIR if needed from $RAW_BACKUP_BASE_DIR/raw_backup_subdir
                                # =False rawspec will read directly from $RAW_BACKUP_BASE_DIR/raw_backup_subdir without file copy

# settings for GBT, override otherwise
if (0):
    fine_fft_size_list = np.array([1])*1024*1024
    n_sti_list = np.array([4])
elif (0):
    fine_fft_size_list = np.array([1])*1024*1024
    n_sti_list = np.array([1])
elif (0):
    fine_fft_size_list = np.array([1])*1024*1024
    n_sti_list = np.array([4,16])
else:
    fine_fft_size_list = np.array([1, 2, 4])*1024*512
    n_sti_list = np.array([1, 4, 16])
        
if (0):
    raw_backup_subdir = 'raw_voyager/'
    raw_file_stem_list = ['blc23_guppi_59046_81310_DIAG_VOYAGER-1_0015'] # 23.7 dB, -.504 Hz/sec
    fig_dir = './voyager_plots/'
    zoom_bw_MHz = .100   # +/- 50 KHz
elif (0):
    raw_backup_subdir = 'raw_voyager/'
    raw_file_stem_list = ['blc3_guppi_57386_VOYAGER1_0004'] 
    fig_dir = './voyager_plots/'
    zoom_bw_MHz = .100   # +/- 50 KHz
    fine_fft_size = 1033216     # =1009*1024, believe it or not
elif (1):
    raw_backup_subdir = 'raw_multichirp/'
    delete_raw_file_enable = False
    raw_file_stem_list = [
        'gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec',
        'gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-45.81sec',
        'gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-91.63sec']
    if (1): raw_file_stem_list.append('gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec')
    if (0): raw_file_stem_list.append('gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-366.50sec')
    fig_dir = './plots/'
    plot_sg_wb = False
    plot_psd_wb = False
    plot_sg_psd_zoom = False
    plot_multichirp_dets2 = True
elif (0):
    raw_backup_subdir = 'raw_voyager/'
    raw_file_stem_list = ['blc23_guppi_59046_80036_DIAG_VOYAGER-1_0011', # 24.4 dB, -.279 Hz/sec
                        'blc23_guppi_59046_80354_DIAG_VOYAGER-1_0012', # no det for 0000
                        'blc23_guppi_59046_80672_DIAG_VOYAGER-1_0013', # 20.93 dB, -.52 Hz/sec
                        'blc23_guppi_59046_80989_DIAG_VOYAGER-1_0014', # no det for 0000
                        'blc23_guppi_59046_81310_DIAG_VOYAGER-1_0015', # 26 dB, -.44 Hz/sec Up to 14 files!
                        'blc23_guppi_59046_81628_DIAG_VOYAGER-1_0016'] # no det for 0000
    fig_dir = './voyager_plots/'
    zoom_bw_MHz = .100   # +/- 50 KHz
elif (1):
    raw_backup_subdir = 'raw_gbt_57388_HIP/'
    raw_file_stem_list = ['blc1_guppi_57388_HIP113357_0010',
                          'blc2_guppi_57388_HIP113357_0010',
                          'blc3_guppi_57388_HIP113357_0010',
                          'blc4_guppi_57388_HIP113357_0010',
                          'blc5_guppi_57388_HIP113357_0010',
                          'blc6_guppi_57388_HIP113357_0010'] 
    fig_dir = './raw_gbt_57388_HIP/'
    fine_fft_size = 1033216     # =1009*1024, believe it or not
elif (0):
    raw_backup_subdir = 'raw_gbt_57388_HIP/'
    raw_file_stem_list = ['blc1_guppi_57388_HIP113357_0012',
                          'blc2_guppi_57388_HIP113357_0012',
                          'blc3_guppi_57388_HIP113357_0012',
                          'blc4_guppi_57388_HIP113357_0012',
                          'blc5_guppi_57388_HIP113357_0012',
                          'blc6_guppi_57388_HIP113357_0012'] 
    fig_dir = './raw_gbt_57388_HIP/'
    fine_fft_size = 1033216     # =1009*1024, believe it or not
elif (0):
    raw_backup_subdir = 'raw_gbt_59103_HIP/'
    raw_file_stem_list = ['blc40_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc41_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc42_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc43_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc44_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc45_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc46_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc47_guppi_59103_03394_DIAG_HIP95631_0015'] 
    fig_dir = './raw_gbt_59103_HIP/'
elif (0):
    raw_backup_subdir = 'raw_gbt_59103_Kepler/'
    raw_file_stem_list = ['blc40_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc41_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc42_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc43_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc44_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc45_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc46_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc47_guppi_59103_01984_DIAG_KEPLER-160_0010'] 
    fig_dir = './raw_gbt_59103_Kepler/'
elif (1):
    raw_backup_subdir = 'raw_gbt_59103_TIC/'
    raw_file_stem_list = ['blc40_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc41_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc42_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc43_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc44_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc45_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc46_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc47_guppi_59103_03721_DIAG_TIC159107668_0016'] 
    fig_dir = './raw_gbt_59103_TIC/'

n_raw_file = len(raw_file_stem_list)

print('raw_file_stem_list=')
for i_raw in range(n_raw_file):
    print(f'{raw_file_stem_list[i_raw]}')
             
print('fine_fft_size_list=',fine_fft_size_list)
print('n_sti_list=',n_sti_list,'\n')

print(f'Figures will appear in in {fig_dir}')

raw_file_stem_list=
gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec
gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-45.81sec
gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-91.63sec
gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec
fine_fft_size_list= [ 524288 1048576 2097152]
n_sti_list= [ 1  4 16] 

Figures will appear in in ./plots/


#### Run end-to-end test over parameter sweeps

In [2]:


for i_raw in range(n_raw_file):
    raw_file_stem = raw_file_stem_list[i_raw]
    print(f'\n\nRaw file {i_raw+1} of {n_raw_file}: {raw_file_stem}\n\n')

    for i_fft, fine_fft_size in enumerate(fine_fft_size_list):
        for i_sti, n_sti in enumerate(n_sti_list):

            try:      
                %run ./01_seti_end_to_end.ipynb
            except:
                print(f'\n\nError Occurred for Raw file {raw_file_stem}, {fine_fft_size=}, {n_sti=}, skip\n\n')

    if delete_raw_file_after_use & (not local_raw_file_found):
        # delete raw file to limit local file storage
        try:
            print('Deleting ',raw_file_delete_spec)
            os.system('rm '+raw_file_delete_spec)
        except:
            print(' ')

            



Raw file 1 of 4: gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec


search_app_name='seticore2', search_app_string='Seticore2', branch='sc2', test_case='sc2-10', search_z_threshold=10
sigma_drift = [0.75 1.5  3.  ]
/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec.0000.raw
Raw file found in raw directory
gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec
['gbt', 'chirp80', '0.0020V', '6002.20', '6003.80', 'm10.0', '10.0', 'Hzsec', '22.91sec']
n_chirp = 80 f_start1_MHz = 6002.2 f_stop1_MHz = 6003.8 sig_min_drift = -10.0 sig_max_drift = 10.0
signal_level = 0.002, n_chirp = 80
f_start1_MHz = 6002.2, f_stop1_MHz = 6003.8,  f_start_incr = 20253.164556503296, df_dt_incr = 0.2531645569620249
f_start_truth[0] = 6002200000.0 f_start_truth[-1] = 6003800000.0
drift_rate_truth[0] = -10.0 drift_rate_truth[-1] = 10.0
raw_file_name='/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec.

dedoppler elapsed time: 2s



Seticore search: 2.16 sec 0.54 sec/coarse channel
E(SNR)    [18.2  17.59 17.06] dB
  1 SNR 29.47 dB,  6000.000 MHz,   0.246 Hz/sec Det Eff  12.3 dB, 1701.7 %
  2 SNR 15.92 dB,  6002.200 MHz,  -9.835 Hz/sec Det Eff  -1.2 dB,  75.2 %
  3 SNR 16.60 dB,  6002.220 MHz,  -9.835 Hz/sec Det Eff  -0.6 dB,  88.0 %
  4 SNR 15.99 dB,  6002.241 MHz,  -9.343 Hz/sec Det Eff  -1.2 dB,  76.5 %
  5 SNR 15.80 dB,  6002.261 MHz,  -9.097 Hz/sec Det Eff  -1.4 dB,  73.1 %
  6 SNR 16.16 dB,  6002.281 MHz,  -8.605 Hz/sec Det Eff  -1.0 dB,  79.4 %
  7 SNR 16.24 dB,  6002.301 MHz,  -8.605 Hz/sec Det Eff  -0.9 dB,  81.0 %
  8 SNR 15.84 dB,  6002.322 MHz,  -8.605 Hz/sec Det Eff  -1.3 dB,  73.9 %
  9 SNR 16.39 dB,  6002.342 MHz,  -8.114 Hz/sec Det Eff  -0.8 dB,  83.7 %
 10 SNR 16.54 dB,  6002.362 MHz,  -7.622 Hz/sec Det Eff  -0.6 dB,  86.8 %
 11 SNR 17.29 dB,  6002.382 MHz,  -7.622 Hz/sec Det Eff   0.1 dB, 103.1 %
 12 SNR 15.95 dB,  6002.403 MHz,  -7.130 Hz/sec Det Eff  -1.2 dB,  75.8 %
 13 SNR 16.24 dB,  6002.423

dedoppler elapsed time: 0s



Seticore search: 0.48 sec 0.12 sec/coarse channel
E(SNR)    [18.23 17.61 17.08] dB
  1 SNR 29.47 dB,  6000.000 MHz,   0.252 Hz/sec Det Eff  12.3 dB, 1701.8 %
  2 SNR 16.04 dB,  6002.220 MHz,  -9.569 Hz/sec Det Eff  -1.1 dB,  77.2 %
  3 SNR 15.81 dB,  6002.240 MHz,  -9.317 Hz/sec Det Eff  -1.3 dB,  73.4 %
  4 SNR 15.58 dB,  6002.261 MHz,  -9.065 Hz/sec Det Eff  -1.6 dB,  69.6 %
  5 SNR 15.92 dB,  6002.281 MHz,  -8.814 Hz/sec Det Eff  -1.2 dB,  75.2 %
  6 SNR 16.15 dB,  6002.301 MHz,  -8.814 Hz/sec Det Eff  -1.0 dB,  79.3 %
  7 SNR 15.84 dB,  6002.322 MHz,  -8.310 Hz/sec Det Eff  -1.3 dB,  73.9 %
  8 SNR 15.97 dB,  6002.342 MHz,  -8.310 Hz/sec Det Eff  -1.2 dB,  76.1 %
  9 SNR 16.54 dB,  6002.362 MHz,  -7.806 Hz/sec Det Eff  -0.6 dB,  86.8 %
 10 SNR 17.29 dB,  6002.382 MHz,  -7.806 Hz/sec Det Eff   0.1 dB, 103.1 %
 11 SNR 15.95 dB,  6002.403 MHz,  -7.303 Hz/sec Det Eff  -1.2 dB,  75.8 %
 12 SNR 16.24 dB,  6002.423 MHz,  -7.303 Hz/sec Det Eff  -0.9 dB,  80.9 %
 13 SNR 16.16 dB,  6002.443

...


Seticore search: 17.18 sec 4.30 sec/coarse channel
E(SNR)    [23.43 23.43 23.19] dB
  1 SNR 19.74 dB,  6002.220 MHz,  -9.765 Hz/sec Det Eff  -4.9 dB,  32.0 %
  2 SNR 19.82 dB,  6002.241 MHz,  -9.482 Hz/sec Det Eff  -4.9 dB,  32.7 %
  3 SNR 19.98 dB,  6002.261 MHz,  -9.245 Hz/sec Det Eff  -4.7 dB,  33.9 %
  4 SNR 19.94 dB,  6002.281 MHz,  -8.985 Hz/sec Det Eff  -4.7 dB,  33.5 %
  5 SNR 20.14 dB,  6002.301 MHz,  -8.732 Hz/sec Det Eff  -4.5 dB,  35.1 %
  6 SNR 20.36 dB,  6002.322 MHz,  -8.480 Hz/sec Det Eff  -4.3 dB,  37.0 %
  7 SNR 20.55 dB,  6002.342 MHz,  -8.235 Hz/sec Det Eff  -4.1 dB,  38.6 %
  8 SNR 20.66 dB,  6002.362 MHz,  -7.967 Hz/sec Det Eff  -4.0 dB,  39.6 %
  9 SNR 20.93 dB,  6002.382 MHz,  -7.730 Hz/sec Det Eff  -3.8 dB,  42.1 %
 10 SNR 20.98 dB,  6002.403 MHz,  -7.477 Hz/sec Det Eff  -3.7 dB,  42.6 %
 11 SNR 21.12 dB,  6002.423 MHz,  -7.217 Hz/sec Det Eff  -3.6 dB,  44.1 %
 12 SNR 21.48 dB,  6002.443 MHz,  -6.964 Hz/sec Det Eff  -3.2 dB,  47.8 %
 13 SNR 21.71 dB,  6002.463

dedoppler elapsed time: 5s



Seticore search: 5.10 sec 1.28 sec/coarse channel
E(SNR)    [22.52 21.15 19.59] dB
  1 SNR 14.12 dB,  6002.220 MHz,  -9.766 Hz/sec Det Eff -10.6 dB,   8.8 %
  2 SNR 14.43 dB,  6002.241 MHz,  -9.495 Hz/sec Det Eff -10.3 dB,   9.4 %
  3 SNR 14.44 dB,  6002.261 MHz,  -9.239 Hz/sec Det Eff -10.2 dB,   9.5 %
  4 SNR 14.39 dB,  6002.281 MHz,  -8.983 Hz/sec Det Eff -10.3 dB,   9.4 %
  5 SNR 14.59 dB,  6002.301 MHz,  -8.736 Hz/sec Det Eff -10.1 dB,   9.8 %
  6 SNR 14.95 dB,  6002.322 MHz,  -8.480 Hz/sec Det Eff  -9.7 dB,  10.6 %
  7 SNR 15.12 dB,  6002.342 MHz,  -8.240 Hz/sec Det Eff  -9.6 dB,  11.0 %
  8 SNR 15.14 dB,  6002.362 MHz,  -7.969 Hz/sec Det Eff  -9.5 dB,  11.1 %
  9 SNR 15.18 dB,  6002.382 MHz,  -7.729 Hz/sec Det Eff  -9.5 dB,  11.2 %
 10 SNR 15.46 dB,  6002.403 MHz,  -7.473 Hz/sec Det Eff  -9.2 dB,  12.0 %
 11 SNR 15.65 dB,  6002.423 MHz,  -7.210 Hz/sec Det Eff  -9.0 dB,  12.5 %
 12 SNR 15.98 dB,  6002.443 MHz,  -6.962 Hz/sec Det Eff  -8.7 dB,  13.5 %
 13 SNR 16.07 dB,  6002.463 

dedoppler elapsed time: 1s



Seticore search: 1.80 sec 0.45 sec/coarse channel
E(SNR)    [19.19 17.05 15.1 ] dB
  1 SNR 10.01 dB,  6002.240 MHz,  -9.473 Hz/sec Det Eff -14.7 dB,   3.4 %
  2 SNR 10.06 dB,  6002.301 MHz,  -8.416 Hz/sec Det Eff -14.6 dB,   3.5 %
  3 SNR 10.33 dB,  6002.321 MHz,  -8.424 Hz/sec Det Eff -14.4 dB,   3.7 %
  4 SNR 10.35 dB,  6002.342 MHz,  -7.733 Hz/sec Det Eff -14.3 dB,   3.7 %
  5 SNR 10.46 dB,  6002.362 MHz,  -8.481 Hz/sec Det Eff -14.2 dB,   3.8 %
  6 SNR 10.42 dB,  6002.382 MHz,  -7.709 Hz/sec Det Eff -14.3 dB,   3.7 %
  7 SNR 10.90 dB,  6002.403 MHz,  -7.473 Hz/sec Det Eff -13.8 dB,   4.2 %
  8 SNR 10.47 dB,  6002.423 MHz,  -7.375 Hz/sec Det Eff -14.2 dB,   3.8 %
  9 SNR 10.38 dB,  6002.443 MHz,  -6.969 Hz/sec Det Eff -14.3 dB,   3.7 %
 10 SNR 11.12 dB,  6002.463 MHz,  -6.709 Hz/sec Det Eff -13.6 dB,   4.4 %
 11 SNR 11.00 dB,  6002.484 MHz,  -6.855 Hz/sec Det Eff -13.7 dB,   4.3 %
 12 SNR 11.74 dB,  6002.504 MHz,  -6.204 Hz/sec Det Eff -12.9 dB,   5.1 %
 13 SNR 11.85 dB,  6002.524 

In [3]:
import os

print('All Runs Done')

# Beep 2 times
for i_beep in range(2):
    try:
        # Beep in WSL
        if os.system("powershell.exe '[console]::beep(261.6,700)'") !=0:
            raise Exception('powershell.exe not found')
    except:
        # linux, probably doesn't work
        print('Beep!')
        os.system("echo -ne '\a'")

All Runs Done
